In [73]:
import py_entitymatching as em
import pandas as pd
import numpy as np

A = em.read_csv_metadata('/Users/daravinds/Documents/Projects/CS-838/Stage3-EntityMatching/CS-838-bookdepository.csv',key='id')
B = em.read_csv_metadata('/Users/daravinds/Documents/Projects/CS-838/Stage3-EntityMatching/CS-838-goodreads.csv',key='id') 

# A = A.replace(np.nan, '', regex=True)
# B = B.replace(np.nan, '', regex=True)
A['Language'] = A['Language'].fillna('')
B['Language'] = B['Language'].fillna('')

A['Publishing Date'] = A['Publishing Date'].fillna('')
B['Publishing Date'] = B['Publishing Date'].fillna('')

A['Publisher'] = A['Publisher'].fillna('')
B['Publisher'] = B['Publisher'].fillna('')

A['Genres'] = A['Genres'].fillna('')
B['Genres'] = B['Genres'].fillna('')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [74]:
I = em.read_csv_metadata("train_set.csv", 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [75]:
# Generate a set of features
F = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [76]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after=['match'])

# Impute feature vectors with the mean of the column values.
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
                strategy='mean')

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [77]:
# Create a set of ML-matchers
lg = em.LogRegMatcher(name='LogReg', random_state=0)

# Train using feature vectors from I 
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')


In [78]:
J = em.read_csv_metadata("after_blocking_tuples.csv", 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

# Convert J into a set of feature vectors using F
L = em.extract_feature_vecs(J, feature_table=F, show_progress=False)
L = L.fillna(0)
em.set_property(L, 'key', '_id')

Metadata file is not present in the given path; proceeding to read the csv file.


True

In [79]:
# Predict on L 
predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

In [80]:
np.count_nonzero(predictions['predicted'])

879

In [81]:
df = pd.DataFrame(predictions)
matches_df = df.loc[df['predicted'] == 1]

match_map = dict()
for index, match in matches_df.iterrows():
    if match['ltable_id'] in match_map:
        existing_matches = match_map.get(match['ltable_id'])
    else:
        existing_matches = set()
    existing_matches.add(match['rtable_id'])
    match_map[match['ltable_id']] = existing_matches


In [82]:
all_values = set()
for values in match_map.values():
    for value in values:
        all_values.add(value)

A_ids = set(A['id'])
A_matched_ids = set(match_map.keys())
ltable_ids_without_match = A_ids.difference(A_matched_ids)

B_ids = set(B['id'])
B_matched_ids = all_values
rtable_ids_without_match = B_ids.difference(B_matched_ids)

merged_data = pd.DataFrame()
data_from_A = pd.DataFrame(A.loc[A['id'].isin(ltable_ids_without_match)], columns=["Title", "Authors", "Genres", "Publishing Date", "Pages", "Publisher", "Language"])
data_from_B = pd.DataFrame(B.loc[B['id'].isin(rtable_ids_without_match)], columns=["Title", "Authors", "Genres", "Publishing Date", "Pages", "Publisher", "Language"])

# print(len(A), len(B))
# print(len(data_from_A), len(data_from_B))
# print(len(ltable_ids_without_match), len(rtable_ids_without_match))
# print(len(match_map), len(all_values))

merged_data = data_from_A.append(data_from_B, ignore_index = True)

In [83]:
# match_map
# for l_id, r_ids in match_map.items():
#     records = list()
#     for r_id in r_ids:
#         records = list()
#         records.append(A.loc[A['id'] == l_id])
#         records.append(B.loc[A['id'] == r_id])
#         final_record = merge_tuples(records)
#         print(final_record)

In [84]:
# def merge_tuples(records):
#     final_title = merge_titles(records)
#     authors = merge_authors(records)
#     genres = merge_genres(records)
#     publishing_date = merge_publishing_dates(records)
#     pages = merge_pages(records)
#     publisher = merge_publishers(records)
#     language = merge_languages(records)
#     return tuple(final_title, authors, genres, publishing_date, pages, publisher, language)

In [85]:
def merge_titles(records):
    length = 0
    for record in records:
        cur_title = record['Title'].strip()
        if len(cur_title) > length:
            length = len(cur_title)
            title = cur_title
#         .to_string(index = False)
#         if title is not None and len(title) > 0:
#             return title
    return title

In [86]:
def merge_authors(records):
    authors = list()
    length = 0
    for record in records:
        current_authors = record['Authors'].strip().split(":")
#         .to_string(index = False).split(":")
        if len(current_authors) > length:
            length = len(current_authors)
            authors = ":".join(current_authors)
    return authors
            
#     A_authors = records[-1]['Authors'].to_string(index = False).split(":")
#     B_authors = records[0]['Authors'].to_string(index = False).split(":")
    
#     if len(A_authors) > len(B_authors):
#         authors = A_authors
#     else:
#         authors = B_authors
#     return ":".join(authors)

In [87]:
def merge_genres(records):
    genres = set(records[-1]['Genres'].strip().split(":"))
#                  .to_string(index = False).split(":"))
    for i in range(len(records) - 1):
        cur_genres = records[i]['Genres'].strip().split(":")
#         .to_string(index = False).split(";")
        genres.union(cur_genres)

    return ":".join(list(genres))

In [88]:
def merge_publishing_dates(records):
    for record in records:
        pub_date = record['Publishing Date'].strip()
#         .to_string(index = False)
        if pub_date is not None and len(pub_date) > 0:
            return pub_date
        
#     pub_date = records[-1]['Publishing Date'].to_string(index = False)
#     if pub_date is None or len(pub_date) == 0:
#         pub_date = records[0]['Publishing Date'].to_string(index = False)
#     return pub_date  

In [89]:
def merge_pages(records):
    for record in records:
        pages = record['Pages'].strip()
#         .to_string(index = False)
        if pages is not None and len(pages) > 0:
            return pages
        
#     pages = records[-1]['Pages'].to_string(index = False)
#     if pages is None or len(pages) == 0:
#         pages = records[0]['Pages'].to_string(index = False)
#     return pages     

In [90]:
def merge_publishers(records):
    for record in records:
        publishers = record['Publisher'].strip()
#         .to_string(index = False)
        if publishers is not None and len(publishers) > 0:
            return publishers
        
#     publishers = records[-1]['Publisher'].to_string(index = False)
#     if publishers is None or len(publishers) == 0:
#         publishers = records[0]['Publisher'].to_string(index = False)
#     return publishers   

In [91]:
def merge_languages(records):
    for record in records:
        languages = record['Language'].strip()
#         .to_string(index = False)
        if languages is not None and len(languages) > 0:
            return languages

#     languages = records[-1]['Language'].to_string(index = False)
#     if languages is None or len(languages) == 0:
#         languages = records[0]['Language'].to_string(index = False)
#     return languages

In [92]:
def merge_tuples(records):
    final_title = merge_titles(records)
    authors = merge_authors(records)
    genres = merge_genres(records)
    publishing_date = merge_publishing_dates(records)
    pages = merge_pages(records)
    publisher = merge_publishers(records)
    language = merge_languages(records)
    final_record = (final_title, authors, genres, publishing_date, pages, publisher, language)

    return final_record

In [93]:
match_map

final_records = list()
for l_id, r_ids in match_map.items():
    records = list()
    matches= B.loc[B['id'].isin(r_ids)]
    for index, match in matches.iterrows():
        records.append(match)
    matches = A.loc[A['id'] == l_id]
    for index, match in matches.iterrows():
        records.append(match)

    record = merge_tuples(records)
    final_records.append(record)
    
final_df = pd.DataFrame(final_records, columns=["Title", "Authors", "Genres", "Publishing Date", "Pages", "Publisher", "Language"])
# final_df
# merged_data
merged_data2 = merged_data.append(final_df, ignore_index = True)
merged_data2.to_csv("unified_table.csv", index=False)

In [95]:
len(merged_data2)

7098